# Importando módulos

Python tiene reglas para la importación de módulos a nuestro código. Un módulo es un archivo cuya extensión es `.py`. Supongamos que tenemos un módulo `rotacion_p.py`.

1. En primer lugar, el intérprete busca un módulo denominado `rotación_p` dentro de los módulos incorporados automáticamente. La lista de dichos módulos se encuentra usando el método `sys.builtin_module_names`:

In [ ]:
import sys 
sys.builtin_module_names

In [ ]:
'rotacion_p' in sys.builtin_module_names

2. En segundo lugar, busca un archivo `rotacion_p.py` en una lista de directorios dada por el atributo `sys.path`. 

In [ ]:
sys.path

Este atributo contiene el directorio local (el primero que aparece en la lista de arriba), y una serie de directorios que provienen de
- La variable de entorno `PYTHONPATH`
- Un directorio dependiente de la instalación (en este caso, '/Users/flavioc/miniconda3/envs/clases/lib/python3.11`).

La manera pythonística de chequear si la variable de entorno `PYTHONPATH` existe es utilizar el método `get` de `os.environ`:

In [ ]:
import os
os.environ.get('PYTHONPATH')

En nuestro caso no imprime nada, pero de la misma forma se puede setear dicha variable de entorno:

In [ ]:
os.environ['PYTHONPATH'] = '..' # seteo la variable PYTHONPATH al directorio padre del directorio actual
os.environ.get('PYTHONPATH')

Por supuesto, todo va a depender de cómo tenemos estructurado nuestro código. En principio, aún cuando uno no utilice completamente las facilidades de Python como lenguaje orientado a objeto, agrupamos funciones que están relacionadas entre sí en distintos módulos. A medida que el código crece, es posible organizar los distintos módulos distribuyéndolos en directorios. 

## Importando módulos de directorios hijos

Imaginemos que tenemos la siguente estructura de código:
```
/miproyecto
    main.py
    /lib
        rotacion.py
    /graficos 
        simple.py
        complejo.py 
        /tresd
        vector.py
```


Es sencillo importar los módulos en los directorios hijos (`lib`, `graficos`):
```python
from graficos.simple import plot_data
from graficos.complejo import plot_data_complejo as plot_complejo 
from graficos.tresd.vector import *
from lib.rotacion import rotate
```

Básicamente exponemos el módulo usando el operador `.` para ir incorporando los hijos. Por ejemplo, `graficos.tresd.vector` refiere al módulo que se encuentra en el archivo `vector.py` en el directorio `graficos.tresd`. 

## Importando módulos desde padres o hermanos

### Imports relativos

Para importar módulos que están en directorios padres o hermanos (estos últimos son directorios al mismo nivel del directorio desde el cual quiero importar) podemos diferentes estrategias. La primera de ellas es usar la importación de paquetes relativos. Para ello, cada directorio desde el que quiera importar debe poseer un archivo (en principio vacío) denominado `__init__.py`. Esto permite a Python reconocer los directorios que contienen módulos aún cuando sean padres o hermanos. 


Veamos ahora la estructura de directorios de `miproyecto_relativo` con los archivos `__init__.py` agregados:
```
/miproyecto
    main.py
    __init__.py
    /lib
        rotacion.py
        __init__.py
    /graficos 
        __init__.py
        simple.py
        complejo.py 
        /tresd
        __init__.py
        vector.py
    /tests
        test_rotacion.py
```

Notemos que `tests/test_rotacion.py` tiene también un `main`, que corre un test:

```python
def test_rotacion():
  v = np.array([1, 0, 0])
  angle = np.array([0, 0, np.pi/2])
  assert np.allclose(rotate(angle, v), np.array([0, -1, 0]))


if __name__ == "__main__":
  test_rotacion()
  print("All tests passed")  
```

Esta es una estructura típica de Python, donde tengo tests que prueban funciones en un módulo dado. 
Notemos cómo se importa el módulo `rotacion` desde `test_rotacion.py`:

```python
from ..lib.rotacion import rotate
```

Al igual que con directorios, `..` se refiere al directorio padre relativo al directorio actual.



Si probamos **desde el directorio** `miproyecto/tests` lo siguiente:

```bash
python test_rotacion.py
```

Nos encontraremos con el error:
```python
ImportError: attempted relative import with no known parent package
```


Para correr el test, tenemos que ir al directorio **padre** del proyecto y correr el main del módulo explícitamente:
```bash
python -m miproyecto_relativo.tests.test_rotacion
All tests passed
```

### Modificando `sys.path`

La forma anterior puede ser engorrosa en el caso en que se tengan muchos módulos en archivos distribuidos en una estructura de directorios complicada.
Por otra parte, es posible modificar el atributo `sys.path` para incluir el directorio que sea de interés. En este caso, modificamos `test_rotacion.py`:

```python
import sys
import os
sys.path.append(os.path.join(os.path.dirname(__file__), ".."))
from lib.rotacion import rotate
```

Entonces, podemos correr desde el directorio `tests`:
```bash
python test_rotacion.py
All tests passed
```
o desde su padre como
```bash
python -m tests.test_rotacion
```
o 
```bash
python tests/test_rotacion.py
```
